# 06 Stability and mismatch proxy E

Section covered: 8 (stability/correctness intuition).

We use `E_proxy = feature_drift + (1 - source_retention_acc)` for transfer runs.

In [ ]:
from pathlib import Path
from copy import deepcopy
import subprocess
import yaml
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path('..').resolve()
LOGS = ROOT / 'outputs' / 'logs'
FIGS = ROOT / 'outputs' / 'figures'
LOGS.mkdir(parents=True, exist_ok=True)
FIGS.mkdir(parents=True, exist_ok=True)

def run_config(config_path: Path, use_progress: bool = True):
    cmd = ['python', str(ROOT / 'scripts' / 'run_transfer.py'), '--config', str(config_path)]
    if use_progress:
        cmd.append('--use-progress')
    subprocess.run(cmd, cwd=ROOT, check=True)

def read_method(name: str) -> pd.DataFrame:
    return pd.read_csv(LOGS / f'transfer_{name}.csv')

def deep_update(base: dict, patch: dict) -> dict:
    for k, v in patch.items():
        if isinstance(v, dict) and isinstance(base.get(k), dict):
            deep_update(base[k], v)
        else:
            base[k] = v
    return base

def apply_fast_dev_profile(cfg: dict) -> dict:
    cfg = deepcopy(cfg)
    data = cfg.setdefault('data', {})
    train = cfg.setdefault('train', {})

    if 'source_train_per_class' in data:
        data['source_train_per_class'] = min(int(data['source_train_per_class']), 220)
    if 'source_test_per_class' in data:
        data['source_test_per_class'] = min(int(data['source_test_per_class']), 80)
    if 'target_train_per_class' in data:
        data['target_train_per_class'] = min(int(data['target_train_per_class']), 30)
    if 'target_test_per_class' in data:
        data['target_test_per_class'] = min(int(data['target_test_per_class']), 80)
    if 'probe_per_class' in data:
        data['probe_per_class'] = min(int(data['probe_per_class']), 40)

    data['batch_size'] = min(int(data.get('batch_size', 128)), 64)
    data['num_workers'] = 0

    if 'source_epochs' in train:
        train['source_epochs'] = min(int(train['source_epochs']), 2)
    if 'target_epochs' in train:
        train['target_epochs'] = min(int(train['target_epochs']), 3)

    train['gradual_schedule'] = {
        '1': ['backbone.layer4'],
        '2': ['backbone.layer3'],
    }
    return cfg

def make_profiled_config(base_name: str, notebook_tag: str, fast_dev_run: bool, overrides: dict | None = None) -> Path:
    cfg = yaml.safe_load((ROOT / 'configs' / base_name).read_text())
    if fast_dev_run:
        cfg = apply_fast_dev_profile(cfg)
    if overrides:
        cfg = deep_update(cfg, deepcopy(overrides))

    suffix = 'fast' if fast_dev_run else 'full'
    out = ROOT / 'configs' / f'tmp_{notebook_tag}_{suffix}.yaml'
    out.write_text(yaml.safe_dump(cfg, sort_keys=False))
    return out


In [ ]:
FAST_DEV_RUN = False
cfg_path = make_profiled_config(
    base_name='transfer_safe_vs_naive.yaml',
    notebook_tag='06_stability',
    fast_dev_run=FAST_DEV_RUN,
)
run_config(cfg_path, use_progress=not FAST_DEV_RUN)


In [ ]:
safe = read_method('gradual_unfreeze').copy()
naive = read_method('naive_finetune').copy()
for df in (safe, naive):
    df['E_proxy'] = df['feature_drift'] + (1.0 - df['source_retention_acc'])

plt.figure(figsize=(6.5, 3.5))
plt.plot(safe['epoch'], safe['E_proxy'], marker='o', label='gradual_unfreeze')
plt.plot(naive['epoch'], naive['E_proxy'], marker='o', label='naive_finetune')
plt.xlabel('epoch')
plt.ylabel('E_proxy')
plt.title('Representation mismatch proxy over adaptation')
plt.grid(alpha=0.2)
plt.legend(frameon=False)


In [ ]:
stability = pd.DataFrame({
    'epoch': safe['epoch'],
    'E_safe': safe['E_proxy'],
    'E_naive': naive['E_proxy'],
    'target_safe': safe['target_test_acc'],
    'target_naive': naive['target_test_acc'],
})
stability.tail()


## Usage

When `E_proxy` increases while target accuracy stalls, lower learning rate or delay unfreezing.